# Import necessary packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error

from timeit import default_timer as timer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [ ]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')

### Note: This Step is for the extraction of California State and Foods category 

In [ ]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [ ]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Foods/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

# User-defined functions to calculate Metrics and Croston algorithm

In [ ]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

In [ ]:
def Croston(ts,extra_periods=1,alpha=0.4):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), periodicity(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    q = 1 #periods since last demand observation
    
    # Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1 + first_occurence
    f[0] = a[0]/p[0]
    # Create all the t+1 forecasts
    for t in range(0,cols):        
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = alpha*q + (1-alpha)*p[t]
            f[t+1] = a[t+1]/p[t+1]
            q = 1           
        else:
            a[t+1] = a[t]
            p[t+1] = p[t]
            f[t+1] = f[t]
            q += 1
       
    # Future Forecast 
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
                      
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

# Set value for parameters

In [ ]:
list_params = [round(item, 1) for item in list(np.arange(0.1, 1, 0.1))]

In [ ]:
list_params

[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
start_train_date = '2014-04-11'
end_train_date = '2016-04-10'
n_pred_days = 14
num_train_needed = 50
validation_training_ratio = 0.95

# Pipeline for Croston

In [ ]:
def Croston_output_all_params(sales_pattern, list_params, start_train_date, end_train_date, 
                       n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    train_data_first_part = train_data.iloc[:int(len(train_data)*validation_training_ratio), :]
    train_data_second_part = train_data.iloc[int(len(train_data)*validation_training_ratio):, :]
    
    df_result_all_params = pd.DataFrame()
    for product in list_pattern_py:
        train_first = train_data_first_part[product]
        train_second = train_data_second_part[product]
        for i in list_params:
            predictions = list()
            history = [x for x in train_first[num_train_needed*-1:]]
            for t in range(len(train_second)):
                yhat = Croston(history, extra_periods=1, alpha=i)['Forecast'].iloc[-1]
                predictions.append(yhat)
                history.append(train_second[t])
            df_result_temp = pd.DataFrame({'Product': [product for count in range(len(train_second))],
                                            'Actual Data': train_second,
                                            'Forecast': predictions,
                                            'Alpha': [i for count in range(len(train_second))]})
            df_result_all_params = df_result_all_params.append(df_result_temp, ignore_index=True)
            
    return df_result_all_params

In [ ]:
start = timer()

pattern_df_result_all_params = Croston_output_all_params(sales_erratic,
                                                      list_params,
                                                      start_train_date,
                                                      end_train_date, 
                                                      n_pred_days,
                                                      num_train_needed,
                                                      validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.9476014935666669 minutes


In [ ]:
pattern_df_result_all_params

,Product,Actual Data,Forecast,Alpha
0,FOODS_1_021_CA_1_validation,7.0,3.406426,0.1
1,FOODS_1_021_CA_1_validation,6.0,3.696428,0.1
2,FOODS_1_021_CA_1_validation,2.0,3.885985,0.1
3,FOODS_1_021_CA_1_validation,3.0,3.727992,0.1
4,FOODS_1_021_CA_1_validation,4.0,3.666001,0.1
...,...,...,...,...
82246,FOODS_3_809_CA_4_validation,0.0,0.986770,0.9
82247,FOODS_3_809_CA_4_validation,0.0,0.986770,0.9
82248,FOODS_3_809_CA_4_validation,0.0,0.986770,0.9
82249,FOODS_3_809_CA_4_validation,1.0,0.986770,0.9


In [ ]:
pattern_df_result_all_params.to_csv('Croston_Erratic_All_Parameters_Hyperparameter_Tuning_Data.csv')

# Summay all metrics

In [ ]:
def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [ ]:
start = timer()

df_result_metrics_all_params = pattern_df_result_all_params.groupby(['Product', 'Alpha']).apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.08693386084999967 minutes


In [ ]:
df_result_metrics_all_params

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_004_CA_2_validation,0.1,0.798667,57.8685,62.8562,83.5139,4.082077,5.035574
1,FOODS_1_004_CA_2_validation,0.2,0.816824,59.1841,63.8251,84.0358,4.174878,5.183997
2,FOODS_1_004_CA_2_validation,0.3,0.832637,60.3298,64.8479,85.2634,4.255700,5.305341
3,FOODS_1_004_CA_2_validation,0.4,0.847604,61.4143,66.1275,86.7769,4.332197,5.399287
4,FOODS_1_004_CA_2_validation,0.5,0.858579,62.2095,67.0292,87.9584,4.388290,5.496496
...,...,...,...,...,...,...,...,...
2218,FOODS_3_827_CA_1_validation,0.5,NaN,NaN,NaN,NaN,0.000000,0.000000
2219,FOODS_3_827_CA_1_validation,0.6,NaN,NaN,NaN,NaN,0.000000,0.000000
2220,FOODS_3_827_CA_1_validation,0.7,NaN,NaN,NaN,NaN,0.000000,0.000000
2221,FOODS_3_827_CA_1_validation,0.8,NaN,NaN,NaN,NaN,0.000000,0.000000


# Check what products has unexpected metrics result

In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MASE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
306,FOODS_1_163_CA_1_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
307,FOODS_1_163_CA_1_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
308,FOODS_1_163_CA_1_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
309,FOODS_1_163_CA_1_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
310,FOODS_1_163_CA_1_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
2218,FOODS_3_827_CA_1_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
2219,FOODS_3_827_CA_1_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
2220,FOODS_3_827_CA_1_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
2221,FOODS_3_827_CA_1_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['WMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
306,FOODS_1_163_CA_1_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
307,FOODS_1_163_CA_1_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
308,FOODS_1_163_CA_1_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
309,FOODS_1_163_CA_1_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
310,FOODS_1_163_CA_1_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
2218,FOODS_3_827_CA_1_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
2219,FOODS_3_827_CA_1_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
2220,FOODS_3_827_CA_1_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
2221,FOODS_3_827_CA_1_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['SMAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
306,FOODS_1_163_CA_1_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
307,FOODS_1_163_CA_1_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
308,FOODS_1_163_CA_1_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
309,FOODS_1_163_CA_1_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
310,FOODS_1_163_CA_1_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
2218,FOODS_3_827_CA_1_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
2219,FOODS_3_827_CA_1_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
2220,FOODS_3_827_CA_1_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
2221,FOODS_3_827_CA_1_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAPE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
306,FOODS_1_163_CA_1_validation,0.1,NaN,NaN,NaN,NaN,0.0,0.0
307,FOODS_1_163_CA_1_validation,0.2,NaN,NaN,NaN,NaN,0.0,0.0
308,FOODS_1_163_CA_1_validation,0.3,NaN,NaN,NaN,NaN,0.0,0.0
309,FOODS_1_163_CA_1_validation,0.4,NaN,NaN,NaN,NaN,0.0,0.0
310,FOODS_1_163_CA_1_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...
2218,FOODS_3_827_CA_1_validation,0.5,NaN,NaN,NaN,NaN,0.0,0.0
2219,FOODS_3_827_CA_1_validation,0.6,NaN,NaN,NaN,NaN,0.0,0.0
2220,FOODS_3_827_CA_1_validation,0.7,NaN,NaN,NaN,NaN,0.0,0.0
2221,FOODS_3_827_CA_1_validation,0.8,NaN,NaN,NaN,NaN,0.0,0.0


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['RMSE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


In [ ]:
df_result_metrics_all_params[df_result_metrics_all_params['MAE'].isin([np.inf, -np.inf, np.nan])]

,Product,Alpha,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE


# Metrics Statistics

In [ ]:
list_metrics = ['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE']

In [ ]:
for mts in list_metrics:
    print('Percentage of unexpected values of', mts, 'is: {}'.format(df_result_metrics_all_params[df_result_metrics_all_params[mts].isin([np.nan, np.inf, -np.inf])].Product.nunique() / df_result_metrics_all_params.Product.nunique() * 100), "%")

Percentage of unexpected values of MASE is: 8.502024291497975 %
Percentage of unexpected values of WMAPE is: 8.502024291497975 %
Percentage of unexpected values of SMAPE is: 9.31174089068826 %
Percentage of unexpected values of MAPE is: 8.502024291497975 %
Percentage of unexpected values of MAE is: 0.0 %
Percentage of unexpected values of RMSE is: 0.0 %


### Filter all rows that have unexpected metrics values

In [ ]:
df_result_metrics_all_params = df_result_metrics_all_params[~df_result_metrics_all_params.isin([np.nan, np.inf, -np.inf]).any(1)]

### Get MEAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.1,0.944699,83.458004,86.529123,74.364575,3.590711,4.660003
0.2,0.916897,80.626949,85.296723,74.861169,3.510929,4.570122
0.3,0.920985,80.791910,85.764568,77.117616,3.538464,4.607599
0.4,0.934256,81.865104,86.606223,79.456940,3.598868,4.685883
0.5,0.952528,83.503253,87.584643,81.783012,3.671079,4.783097
0.6,0.974644,85.566134,88.705394,84.225592,3.751248,4.895039
0.7,0.998197,87.806260,89.873667,86.723858,3.834853,5.023188
0.8,1.023194,90.183977,91.183898,89.416914,3.925229,5.171712
0.9,1.050861,92.831962,92.732963,92.588370,4.032419,5.348196


In [ ]:
for mts in list_metrics:
    print('The optimum Alpha based on Mean', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].mean()[mts].idxmin()))

The optimum Alpha based on Mean MASE is: 0.2
The optimum Alpha based on Mean WMAPE is: 0.2
The optimum Alpha based on Mean SMAPE is: 0.2
The optimum Alpha based on Mean MAPE is: 0.1
The optimum Alpha based on Mean MAE is: 0.2
The optimum Alpha based on Mean RMSE is: 0.2


### Get MEDIAN metrics value of each Alpha

In [ ]:
df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()

,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
Alpha,,,,,,
0.1,0.818583,69.2048,78.9160,67.6372,2.584797,3.306286
0.2,0.813046,68.5433,76.9832,68.5591,2.577061,3.250311
0.3,0.832637,70.1005,77.8462,70.4057,2.643960,3.355981
0.4,0.848228,70.8839,78.8813,72.8081,2.722958,3.407778
0.5,0.864574,72.8071,79.6044,74.8552,2.777182,3.499459
0.6,0.893163,74.8981,81.4175,77.7666,2.823545,3.522283
0.7,0.913796,75.9046,83.1509,79.3763,2.888439,3.547848
0.8,0.937136,78.6827,84.3447,80.8277,2.989062,3.670340
0.9,0.962428,80.4760,86.0419,84.3314,3.034022,3.854678


In [ ]:
for mts in list_metrics:
    print('The optimum Alpha based on Median of', mts, 'is: {}'.format(df_result_metrics_all_params.groupby(['Alpha'])['MASE', 'WMAPE', 'SMAPE', 'MAPE', 'MAE', 'RMSE'].median()[mts].idxmin()))

The optimum Alpha based on Median of MASE is: 0.2
The optimum Alpha based on Median of WMAPE is: 0.2
The optimum Alpha based on Median of SMAPE is: 0.2
The optimum Alpha based on Median of MAPE is: 0.1
The optimum Alpha based on Median of MAE is: 0.2
The optimum Alpha based on Median of RMSE is: 0.2


# Run Croston model after decide best paramaters

In [ ]:
best_alpha = 0.2

In [ ]:
def Croston_test_data(best_alpha, sales_pattern, start_train_date, end_train_date, 
                       n_pred_days, num_train_needed, validation_training_ratio):
    
    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()
    
    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')
    
    train_data = df_pattern[(df_pattern.index >= start_train_date) & (df_pattern.index <= end_train_date)]
    test_data = df_pattern.iloc[n_pred_days*-1:]
    
    df_result_best_params = pd.DataFrame()
    for product in list_pattern_py:
        train_product = train_data[product]
        test_product = test_data[product]
        predictions = list()
        history = [x for x in train_product[num_train_needed*-1:]]
        for t in range(len(test_product)):
            yhat = Croston(history, extra_periods=1, alpha=best_alpha)['Forecast'].iloc[-1]
            predictions.append(yhat)
            history.append(test_product[t])
        df_result_temp = pd.DataFrame({'Product': [product for count in range(len(test_product))],
                                        'Actual Data': test_product,
                                        'Forecast': predictions})
        df_result_best_params = df_result_best_params.append(df_result_temp, ignore_index=True)
            
    return df_result_best_params

In [ ]:
start = timer()

pattern_df_result_best_params = Croston_test_data(best_alpha, sales_erratic, start_train_date, end_train_date, n_pred_days, num_train_needed, validation_training_ratio)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.04617038768333259 minutes


In [ ]:
df_result_final = pattern_df_result_best_params.groupby('Product').apply(get_metrics_result_all_params).reset_index()

In [ ]:
df_result_final

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,FOODS_1_004_CA_2_validation,1.012094,59.5838,66.1123,100.0527,3.191989,3.652494
1,FOODS_1_018_CA_2_validation,1.168533,82.2813,77.1999,133.1548,3.056162,3.518693
2,FOODS_1_018_CA_3_validation,0.807508,63.2454,65.9768,95.1606,2.484640,3.234042
3,FOODS_1_021_CA_1_validation,0.954901,82.0235,86.1864,159.5221,4.921411,5.965742
4,FOODS_1_021_CA_2_validation,0.913224,63.5159,65.1676,70.7022,2.177689,2.731308
...,...,...,...,...,...,...,...
242,FOODS_3_809_CA_4_validation,0.637197,102.9319,123.6038,34.6375,0.735228,0.916500
243,FOODS_3_811_CA_2_validation,0.973148,71.9526,88.3108,44.1728,3.443445,3.971165
244,FOODS_3_818_CA_2_validation,1.279985,109.3250,134.8358,69.2069,2.264590,2.864551
245,FOODS_3_822_CA_3_validation,0.684961,37.2969,41.2145,28.6306,2.371019,3.029145


In [ ]:
df_result_final.isnull().sum()

Product     0
MASE       19
WMAPE      19
SMAPE      20
MAPE       24
MAE         0
RMSE        0
dtype: int64

In [ ]:
df_result_final.to_csv('Croston_Erratic_Test_Data.csv')